In [25]:
from moviepy.editor import ImageSequenceClip
from PIL import Image, ImageDraw, ImageFont
import os
import re

In [26]:
code_lines = ["def my_fun(a, b):", "      print(f'it is a test {a}')"]

In [27]:
# Font settings (Adjust the path to your font file)
font_path = "./fonts/ARLRDBD.ttf"  # Adjust this path to the font you want to use
font_size = 70
font = ImageFont.truetype(font_path, font_size)

In [28]:
def preprocess_line(text):
      words = []

      pattern = r'(\(|\))'
      words = re.split(pattern, text)
      result = []
      for word in words:
            if "'" not in word and "\"" not in word:
                  if " " in word:
                        pattern = r'(\s+)'
                        temp = re.split(pattern, word)
                        result += temp
                  else:
                        result.append(word)
            else:
                  if word.startswith("f"):                                              
                        result += ["f", word[1:]]
                  else:
                        result.append(word)

      return result

In [29]:
# Simulate basic syntax highlighting for Python code
def syntax_highlight(text):
    keywords = ["print", "def", "for", "in", "return", "if", "else"]
    colors = {"keyword": "blue", "string": "green", "default": "black"}
    
    highlighted_text = []
    words = preprocess_line(text)
    for word in words:
        color = colors["default"]
        if word in keywords:
            color = colors["keyword"]
        elif word.startswith("'") or word.startswith('"'):
            color = colors["string"]
        highlighted_text.append((word, color))
    return highlighted_text

In [30]:
def draw_rounded_rectangle(draw, xy, corner_radius, fill=None, outline=None):
    upper_left_point = (xy[0], xy[1])
    bottom_right_point = (xy[2], xy[3])

    # Draw the four corners
    draw.pieslice([upper_left_point, (upper_left_point[0] + corner_radius * 2, upper_left_point[1] + corner_radius * 2)], start=180, end=270, fill=fill, outline=outline)
    draw.pieslice([(bottom_right_point[0] - corner_radius * 2, upper_left_point[1]), (bottom_right_point[0], upper_left_point[1] + corner_radius * 2)], start=270, end=360, fill=fill, outline=outline)
    draw.pieslice([(upper_left_point[0], bottom_right_point[1] - corner_radius * 2), (upper_left_point[0] + corner_radius * 2, bottom_right_point[1])], start=90, end=180, fill=fill, outline=outline)
    draw.pieslice([(bottom_right_point[0] - corner_radius * 2, bottom_right_point[1] - corner_radius * 2), bottom_right_point], start=0, end=90, fill=fill, outline=outline)

    # Draw the four sides
    draw.rectangle([upper_left_point[0], upper_left_point[1] + corner_radius, upper_left_point[0] + corner_radius, bottom_right_point[1] - corner_radius], fill=fill)
    draw.rectangle([bottom_right_point[0] - corner_radius, upper_left_point[1] + corner_radius, bottom_right_point[0], bottom_right_point[1] - corner_radius], fill=fill)
    draw.rectangle([upper_left_point[0] + corner_radius, upper_left_point[1], bottom_right_point[0] - corner_radius, upper_left_point[1] + corner_radius], fill=fill)
    draw.rectangle([upper_left_point[0] + corner_radius, bottom_right_point[1] - corner_radius, bottom_right_point[0] - corner_radius, bottom_right_point[1]], fill=fill)

    # Draw the center rectangle
    draw.rectangle([upper_left_point[0] + corner_radius, upper_left_point[1] + corner_radius, bottom_right_point[0] - corner_radius, bottom_right_point[1] - corner_radius], fill=fill)


In [46]:
# Image settings
img_width, img_height = 1080, 1920
background_color = "white"
rect_background_color = (128, 128, 128)
rect_border_color = (0, 0, 0)
padding = 50  # Padding around the rectangle
rectangle_height = 300

In [47]:
# Generate a sequence of images
images = []
y_init = 0 + padding * 1.25
bkg_image = None
for code_line in code_lines:
    for i in range(1, len(code_line) + 1):
        if bkg_image:
            img = bkg_image.copy()
        else:
            img = Image.new("RGB", (img_width, img_height), color=background_color)
        draw = ImageDraw.Draw(img)
        
        # Draw a rectangle container
        if bkg_image is None:
            draw_rounded_rectangle(draw, [padding, padding, img_width - padding, rectangle_height], 20, outline=rect_border_color, fill=rect_background_color)
        
        # Apply simplified syntax highlighting
        partial_code = syntax_highlight(code_line[:i])       
        x, y = 20 + padding * 1.25, 30 + y_init  
        for word, color in partial_code:
            draw.text((x, y_init), word, fill=color, font=font)
            word_width = draw.textlength(word + " ", font=font)  # Correct method to get text size
            word_height = font_size
            x += word_width  # Move x for the next word
        images.append(img)
    y_init += font_size * 1.2
    bkg_image = img.copy()

In [48]:
# Save images to a temporary directory
temp_dir = "temp_images"
os.makedirs(temp_dir, exist_ok=True)
filenames = []
for idx, img in enumerate(images):
    filename = os.path.join(temp_dir, f"{idx}.png")
    img.save(filename)
    filenames.append(filename)

In [49]:
# Create a video clip from images
clip = ImageSequenceClip(filenames, fps=5)  # Adjust fps for typing speed

# Export the video
output_video = "typing_effect.mp4"
clip.write_videofile(output_video, codec="libx264")

# Cleanup: remove temporary images
for filename in filenames:
    os.remove(filename)
os.rmdir(temp_dir)

print(f"Video saved as {output_video}")

Moviepy - Building video typing_effect.mp4.
Moviepy - Writing video typing_effect.mp4



Moviepy - Done !
Moviepy - video ready typing_effect.mp4
Video saved as typing_effect.mp4
